In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [ ]:
class KMaxPooling(nn.Module):
    """
    K-Max Pooling used as the last layer for VDCNN model implementation
    """
    def __init__(self, k=8):
        super(KMaxPooling, self).__init__()
        self.k = 8
    def forward(self, x):
        idxes = x.topk(self.k, dim=2)[1].sort(dim=0)[0]
        return x.gather(2, idxes)


class VDCNN(nn.Module):
    """
    Implmentation of VDCNN model by Conneau et al. 2017
    """
    def __init__(self, depth=9, num_class=10):
        super(VDCNN, self).__init__()
        self.depth = depth
        self.num_class = num_class
        
        if depth == 9:
            num_blocks = [2, 2, 2, 2]
        elif depth == 17:
            num_blocks = [4, 4, 4, 4]
        elif depth == 29:
            num_blocks = [4, 4, 10, 10]
        else:
            print('Choose depth among 9, 17, or 29.')
            sys.exit()

        self.conv64_block = nn.ModuleList()
        self.conv64_block += [nn.Conv1d(16, 64, 3, padding=2)]
        for i in range(num_blocks[0]):
            self.conv64_block += [nn.Conv1d(64, 64, 3, padding=2), 
                                  nn.BatchNorm1d(64, affine=False),
                                  nn.ReLU()]

        self.pool_half = nn.MaxPool1d(3, stride=2) 
        
        self.conv128_block = nn.ModuleList()
        for i in range(num_blocks[1]):
            if i == 0:
                self.conv128_block += [nn.Conv1d(64, 128, 3, padding=2)]
            else:
                self.conv128_block += [nn.Conv1d(128, 128, 3, padding=2)]
            self.conv128_block += [nn.BatchNorm1d(128, affine=False),
                                   nn.ReLU()]   
        
        self.conv256_block = nn.ModuleList()
        for i in range(num_blocks[2]):
            if i == 0:
                self.conv256_block += [nn.Conv1d(128, 256, 3, padding=2)]
            else:
                self.conv256_block += [nn.Conv1d(256, 256, 3, padding=2)]
            self.conv256_block += [nn.BatchNorm1d(256, affine=False),
                                   nn.ReLU()]
        
        self.conv512_block = nn.ModuleList()
        for i in range(num_blocks[3]):
            if i == 0:
                self.conv256_block += [nn.Conv1d(256, 512, 3, padding=2)]
            else:
                self.conv256_block += [nn.Conv1d(512, 512, 3, padding=2)]
            self.conv256_block += [nn.BatchNorm1d(512, affine=False),
                                   nn.ReLU()]
        
        self.output_block = nn.ModuleList()
        
        self.output_block += [KMaxPooling(k=8),
                              nn.Linear(4096, 2048),
                              nn.ReLU(),
                              nn.Linear(2048, 2048),
                              nn.ReLU(),
                              nn.Linear(2048, self.num_class),
                              nn.Softmax(dim=1)]
        
    def forward(self, x):
        for unit in self.conv64_block:
            x = unit(x)
        x = self.pool_half(x)
        for unit in self.conv128_block:
            x = unit(x)
        x = self.pool_half(x)
        for unit in self.conv256_block:
            x = unit(x)
        x = self.pool_half(x)
        for unit in self.conv512_block:
            x = unit(x)
        for unit in self.output_block:
            x = unit(x)
        return x
    
    

        

In [8]:
train_df = pd.read_csv('../../data/yelp_review_polarity_csv/train.csv', header=None)
test_df = pd.read_csv('../../data/yelp_review_polarity_csv/test.csv', header=None)

In [39]:
lookup_table = {}
chrs = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'"+'"/\|_@#$%^&*~`+-=<>()[]{} '
for i, c in enumerate(chrs):
    lookup_table[c] = i+1 # reserve 0 as padding

train_X = np.zeros([len(train_df), 1024]).astype(int)
for i, s in enumerate(train_df.loc[:, 1]):
    for j, c in enumerate(s.lower()):
        train_X[i, j] = lookup_table[c]
        if j == 1023:
            break

train_Y = train_df.loc[:, 0].to_numpy().reshape(-1, 1)


array([[21, 14,  6, ...,  0,  0,  0],
       [ 2,  5,  5, ...,  0,  0,  0],
       [ 9, 70,  4, ...,  5, 12, 20],
       ...,
       [16,  8, 15, ...,  0,  0,  0],
       [12, 15, 15, ...,  0,  0,  0],
       [18, 25,  1, ...,  0,  0,  0]])